In [1]:
import pandas as pd
import numpy as np
from pymoo.optimize import minimize
import pickle

from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination
from pymoo.visualization.scatter import Scatter
from notebooks.optimization_problems.constraints import Requirements
from pyreport import PlotUtil
import matplotlib.pyplot as plt

%load_ext autoreload

%autoreload 2

In [2]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z',
           'd',
           'r_ab_sff_x', 'r_ab_sff_y', 'r_ab_sff_z']

file_path = "example_data.h5"

row_limit = -1

store_500km = pd.HDFStore(file_path)

instances_500km_df = store_500km.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances_500km_df = instances_500km_df.sort_index(0)

In [3]:
N_passes = 20

instances_df = instances_500km_df.loc[0:N_passes,:]

In [110]:
from notebooks.optimization_problems.pointing_problem import PointingProblem
from notebooks.optimization_problems.design_vector import design_vector_default_scm, SystemParameters

sys_param = SystemParameters()
sys_param.margin_dB = 3.0
sys_param.B_Hz_array = np.array([0.1, 0.5, 1, 10, 50, 100, 200, 300]) * 1e6
sys_param.Gtx_dBi_bounds = (3., 30.)
sys_param.Ptx_dBm_bounds = (20., 43.)

modcods_df = pd.read_pickle('dvbs2.pkl')

sys_param.EsN0_req_dB_array = modcods_df['isend'].to_numpy()
sys_param.eta_bitsym_array = modcods_df[['eta', 'eta_200MHz', 'eta_300MHz']].to_numpy()
sys_param.eta_maee_array = modcods_df[['maee_12', 'maee_12_200MHz', 'maee_12_300MHz']].to_numpy()

requirements = Requirements()
requirements.min_throughput = 0.5e9
#requirements.max_throughput = 5e9

problem = PointingProblem(instances_df, sys_param, requirements=requirements)

sampling, crossover, mutation = design_vector_default_scm(problem.x_length, problem.x_indices)

algo_settings = {'pop_size': 400, 'n_offsprings': 20}

settings = {}

# NGSA-II
from pymoo.algorithms.nsga2 import NSGA2
settings['NGSA-II'] = {
    'label': 'ngsa2',
    'algorithm': NSGA2(
        pop_size=algo_settings['pop_size'],
        n_offsprings=algo_settings['n_offsprings'],
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        eliminate_duplicates=True,
    ),
    'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
                                                      n_last=30,
                                                      nth_gen=5,
                                                      n_max_gen=20000,
                                                      n_max_evals=None)
}

# NGSA-III
from pymoo.algorithms.nsga3 import NSGA3
from pymoo.factory import get_reference_directions, get_visualization

ref_dirs = get_reference_directions("das-dennis", 3, n_partitions=25)
#get_visualization("scatter").add(ref_dirs).show()
settings['NGSA-III'] = {
    'label': 'ngsa3',
    'algorithm': NSGA3(
        pop_size=algo_settings['pop_size'],
        n_offsprings=algo_settings['n_offsprings'],
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        ref_dirs=ref_dirs,
        eliminate_duplicates=True,
    ),
    'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
                                                      n_last=30,
                                                      nth_gen=5,
                                                      n_max_gen=20000,
                                                      n_max_evals=None)
}

# R-NGSA-II
from pymoo.algorithms.rnsga2 import RNSGA2
ref_points = np.array([
    [0, 1, 1], # Max throughput, d.c. energy, d.c. pointing
    [0, 0, 1], # Max throughput, min energy, d.c. pointing
    [0, 1, 0], # Max throughput, d.c. energy, min pointing
    [1, 0, 0], # d.c. f_throughput, min energy, min pointing
])
settings['RNGSA-II'] = {
    'label': 'rngsa2',
    'algorithm': RNSGA2(
        pop_size=algo_settings['pop_size'],
        n_offsprings=algo_settings['n_offsprings'],
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        eliminate_duplicates=True,
        ref_points=ref_points,
        epsilon=0.001,
        normalization='front',
        extreme_points_as_reference_points=False,
        #weights=np.array([0.5, 0.5, 0.5])
    ),
    'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
                                                      n_last=30,
                                                      nth_gen=5,
                                                      n_max_gen=5000,
                                                      n_max_evals=None)
}

setting = settings['NGSA-II']
setting = settings['NGSA-III']
setting = settings['RNGSA-II']

In [ ]:
termination = setting['termination']
algorithm = setting['algorithm']

res = minimize(problem,
               algorithm,
               termination=termination,
               seed=1,
               #save_history=True,
               verbose=True
               )

print('Processes:', res.exec_time)
print("Best solution found: %s" % res.X)

n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |     400 |  0.00000E+00 |  3.03008E+07 |      15 |            - |            -
    2 |     420 |  0.00000E+00 |  8.66701E+06 |      18 |  0.015350141 |        nadir
    3 |     440 |  0.00000E+00 |  1.23808E+06 |      18 |  0.00000E+00 |            f
    4 |     460 |  0.00000E+00 |  1.73448E+02 |      18 |  0.00000E+00 |            f
    5 |     480 |  0.00000E+00 |  1.63388E+02 |      18 |  0.011552673 |            f
    6 |     500 |  0.00000E+00 |  1.48293E+02 |      21 |  0.007199987 |            f
    7 |     520 |  0.00000E+00 |  1.34863E+02 |      24 |  0.011687693 |            f
    8 |     540 |  0.00000E+00 |  1.24893E+02 |      24 |  0.00000E+00 |            f
    9 |     560 |  0.00000E+00 |  1.15703E+02 |      26 |  0.007242931 |            f
   10 |     580 |  0.00000E+00 |  1.07910E+02 |      27 |  0.003126419 |        ideal
   11 |     600 |  0.00000E+00 |  1.00258E+02 |      2

In [97]:
pickle.dump(res, open('pointing_res_%s.pkl' % (setting['label']), 'wb'))

In [98]:
#res = pickle.load(open('pointing_res_ngsa2.pkl' % (setting['label']), 'rb'))
#res = pickle.load(open('pointing_res_ngsa3.pkl' % (setting['label']), 'rb'))
res = pickle.load(open('pointing_res_%s.pkl' % (setting['label']), 'rb'))

In [99]:
x_pass = res.X[:, problem.x_indices['pass']].astype('bool')
x_Ptx_dBm = res.X[:, problem.x_indices['power']].astype('float64')
x_Ptx_dBm[~x_pass] = np.NaN
x_Ptx_dBm = np.nanmax(x_Ptx_dBm, axis=1)
#x_modcod = np.squeeze(res.X[:, problem.x_indices['modcod']].astype('int64'))

f_pointing = res.F[:, 2]
f_energy = res.F[:,1] / 1e3 # Kilo Joule
f_throughput = ((res.F[:,0] * -1) / 1e9)   # Gigabit

In [106]:
i_maxT = np.argmin(np.linalg.norm(res.F / np.max(np.abs(res.F), axis=0) - ref_points[0,:], axis=1))
i_maxTminE = np.argmin(np.linalg.norm(res.F / np.max(np.abs(res.F), axis=0) - ref_points[1,:], axis=1))
i_maxTminP = np.argmin(np.linalg.norm(res.F / np.max(np.abs(res.F), axis=0) - ref_points[2,:], axis=1))
i_minEminP = np.argmin(np.linalg.norm(res.F / np.max(np.abs(res.F), axis=0)- ref_points[3,:], axis=1))

p_maxT = np.array([f_energy[i_maxT], f_pointing[i_maxT], f_throughput[i_maxT]])
p_maxTminE = np.array([f_energy[i_maxTminE], f_pointing[i_maxTminE], f_throughput[i_maxTminE]])
p_maxTminP = np.array([f_energy[i_maxTminP], f_pointing[i_maxTminP], f_throughput[i_maxTminP]])
p_minEminP = np.array([f_energy[i_minEminP], f_pointing[i_minEminP], f_throughput[i_minEminP]])

# i_max = np.argmax(f_throughput)
# i_min = np.argmin(f_pointing)
# i_pick = np.argmin(np.abs(f_throughput - 1000))
#
# if setting == settings['NGSA-II']:
#     i_pick = 358
# if setting == settings['NGSA-III']:
#     i_pick = 44
#
# p_max_T = (f_pointing[i_max], f_energy[i_max], f_throughput[i_max])
# p_min_P = (f_pointing[i_min], f_energy[i_min], f_throughput[i_min])
# p_pick_P = (f_pointing[i_pick], f_energy[i_pick], f_throughput[i_pick])

In [108]:
%matplotlib

fig = plt.figure(figsize=(3.2, 2.4))
ax = fig.add_subplot(111, projection='3d', proj_type = 'ortho')

ax.scatter(f_pointing, f_energy, f_throughput, color='#1f77b4', marker='.', s=1)
ax.scatter(p_maxT[0], p_maxT[1], p_maxT[2], color='r', s=10)
ax.scatter(p_maxTminE[0], p_maxTminE[1], p_maxTminE[2], color='g', s=10)
ax.scatter(p_maxTminP[0], p_maxTminP[1], p_maxTminP[2], color='b', s=10)
ax.scatter(p_minEminP[0], p_minEminP[1], p_minEminP[2], color='y', s=10)
# ax.scatter(p_pick_P[0], p_pick_P[1], color='y')

ax.set_ylabel("Energy used [kJ]")
ax.set_zlabel("Throughput [Gbit]")
ax.set_xlabel("Pointing [s]")

#ax.invert_xaxis()
#ax.invert_yaxis()

PlotUtil.apply_report_formatting_3D()
plt.tight_layout(pad=1.5, rect=(-0.1, -0.05, 1.15, 1.15))
ax.view_init(15, -140)

plt.ion()
#plt.show()

fig.savefig('D:/git/thesis_report_ae/figures/pointing_pareto_3d_%s.pdf' % setting['label'])

Using matplotlib backend: Qt5Agg


In [136]:
print('ax.azim {}'.format(ax.azim))
print('ax.elev {}'.format(ax.elev))

ax.azim -129.52638700947202
ax.elev 17.922868741542743


In [109]:
fig = plt.figure(figsize=(3.2*3, 2.4))
axs = fig.subplots(1,3)

ax = axs[0]
ax.grid(True)
ax.scatter(f_energy, f_throughput, marker='.', s=1)
ax.scatter(p_maxT[0], p_maxT[2], color='r', s=10)
ax.scatter(p_maxTminE[0], p_maxTminE[2], color='g', s=10)
ax.scatter(p_maxTminP[0], p_maxTminP[2], color='b', s=10)
ax.scatter(p_minEminP[0], p_minEminP[2], color='y', s=10)

# for d in np.arange(len(f_throughput)):
#     ax.text(f_energy[d], f_throughput[d], '%d' % d)

ax.set_xlabel("Energy used [kJ]")
ax.set_ylabel("Throughput [Gbit]")

ax = axs[1]
ax.grid(True)
ax.scatter(f_pointing, f_throughput, marker='.', s=1)
ax.scatter(p_maxT[1], p_maxT[2], color='r', s=10)
ax.scatter(p_maxTminE[1], p_maxTminE[2], color='g', s=10)
ax.scatter(p_maxTminP[1], p_maxTminP[2], color='b', s=10)
ax.scatter(p_minEminP[1], p_minEminP[2], color='y', s=10)
ax.set_xlabel("Pointing [s]")
ax.set_ylabel("Throughput [Gbit]")

ax = axs[2]
ax.grid(True)
ax.scatter(f_energy, f_pointing, marker='.', s=1)
ax.scatter(p_maxT[0], p_maxT[1], color='r', s=10)
ax.scatter(p_maxTminE[0], p_maxTminE[1], color='g', s=10)
ax.scatter(p_maxTminP[0], p_maxTminP[1], color='b', s=10)
ax.scatter(p_minEminP[0], p_minEminP[1], color='y', s=10)
ax.set_ylabel("Pointing [s]")
ax.set_xlabel("Energy used [kJ]")

plt.tight_layout()

fig.savefig('D:/git/thesis_report_ae/figures/pointing_pareto_tri_%s.pdf' % setting['label'])

In [51]:
x_bandwidth = res.X[:, problem.x_indices['bandwidth']].astype('int')
B_Hz = sys_param.B_Hz_array[np.squeeze(x_bandwidth)] / 1e6

fig = plt.figure(figsize=(3.2, 2.4))

ax = fig.add_subplot(1,1,1)
ax.grid()
ax.set_axisbelow(True)

ax.scatter(B_Hz, f_throughput, marker='.', s=2)
#ax.set_xscale('log')
ax.set_xlabel("Bandwidth [MHz]")
ax.set_ylabel("Throughput [Gbit]")
#ax.set_xlim((, 11))
ax.set_ylim((0, 1200))

PlotUtil.apply_report_formatting(3.146, 2.76)
plt.tight_layout()

#fig.savefig('D:/git/thesis_report_ae/figures/vcm_modcod_vs_throughput.pdf')

In [29]:
fig = plt.figure(figsize=(3.2, 2.4))

ax = fig.add_subplot(1,1,1)
ax.grid()
ax.set_axisbelow(True)

ax.scatter(np.sum(x_pass, axis=1), f_throughput, marker='.', s=1)
ax.set_xlabel("Number of passes used")
ax.set_ylabel("Throughput [Gbit]")
ax.set_xlim((0, 20))
ax.set_ylim((0, 1200))

PlotUtil.apply_report_formatting(3.146, 2.76)
plt.tight_layout()

#fig.savefig('D:/git/thesis_report_ae/figures/vcm_modcod_vs_throughput.pdf')

In [30]:
fig = plt.figure(figsize=(3.2, 2.4))

import matplotlib.pyplot as plt
from pymoo.performance_indicator.hv import Hypervolume

# create the performance indicator object with reference point (4,4)
metric = Hypervolume(ref_point=np.array([0,1e18]))

# collect the population in each generation
pop_each_gen = [a.pop for a in res.history]

# receive the population in each generation
obj_and_feasible_each_gen = [pop[pop.get("feasible")[:,0]].get("F") for pop in pop_each_gen]

# calculate for each generation the HV metric
hv = [metric.calc(f) for f in obj_and_feasible_each_gen]

# function evaluations at each snapshot
n_evals = np.array([a.evaluator.n_eval for a in res.history])

# visualze the convergence curve
plt.plot(n_evals, hv, '-o', markersize=3)
plt.title("Convergence")
plt.xlabel("Function Evaluations")
plt.ylabel("Hypervolume")
PlotUtil.apply_report_formatting()
plt.tight_layout()
plt.grid()
#plt.savefig('D:/git/thesis_report_ae/figures/link_budget_hypervolume%s.svg' % postfix)
fig.savefig('D:/git/thesis_report_ae/figures/visibility_hypervolume%s.pdf' % postfix)
plt.show()
plt.close()

TypeError: 'NoneType' object is not iterable

In [72]:
fig = plt.figure(figsize=(3.2, 2.4))
ax = fig.add_subplot(111, projection='3d', proj_type = 'ortho')

ax.scatter(ref_dirs[:,1], ref_dirs[:,2], 1-ref_dirs[:,0], color='#1f77b4', marker='.', s=10)

ax.set_ylabel("Energy used [kJ]")
ax.set_zlabel("Throughput [Gbit]")
ax.set_xlabel("Pointing [s]")

#ax.invert_xaxis()
#ax.invert_yaxis()

PlotUtil.apply_report_formatting_3D()
plt.tight_layout(pad=1.5, rect=(-0.1, -0.05, 1.15, 1.15))
ax.view_init(20, -135)

plt.ion()
#plt.show()

fig.savefig('D:/git/thesis_report_ae/figures/pointing_reference_dirs.pdf')

In [71]:
print('ax.azim {}'.format(ax.azim))
print('ax.elev {}'.format(ax.elev))


ax.azim -136.94857916102848
ax.elev 19.269282814614144


In [93]:
fig = plt.figure(figsize=(3.2, 2.4))
ax = fig.add_subplot(111, projection='3d', proj_type = 'ortho')

ref_points_plt = np.abs(ref_points / np.max(ref_points))
ax.scatter(ref_points_plt[0,1], ref_points_plt[0,2], 1-ref_points_plt[0,0], color='r', marker='.', s=20)
ax.scatter(ref_points_plt[1,1], ref_points_plt[1,2], 1-ref_points_plt[1,0], color='g', marker='.', s=20)
ax.scatter(ref_points_plt[2,1], ref_points_plt[2,2], 1-ref_points_plt[2,0], color='b', marker='.', s=20)
ax.scatter(ref_points_plt[3,1], ref_points_plt[3,2], 1-ref_points_plt[3,0], color='y', marker='.', s=20)

ax.set_ylabel("Energy used [kJ]")
ax.set_zlabel("Throughput [Gbit]")
ax.set_xlabel("Pointing [s]")
# ax.set_xlim(0,1)
# ax.set_ylim(0,1)
# ax.set_zlim(0,1)


#ax.invert_xaxis()
#ax.invert_yaxis()

PlotUtil.apply_report_formatting_3D()
plt.tight_layout(pad=1.5, rect=(-0.1, -0.05, 1.15, 1.15))
ax.view_init(20, -135)

plt.ion()
#plt.show()

fig.savefig('D:/git/thesis_report_ae/figures/pointing_reference_points.pdf')

In [ ]:
print('ax.azim {}'.format(ax.azim))
print('ax.elev {}'.format(ax.elev))

In [68]:
%matplotlib

plt.close()

Using matplotlib backend: Qt5Agg
